Setup

In [1]:
import pandas as pd
import numpy as np
import string
import seaborn as sns
!pip install -U scikit-multiflow
#resource for profanity: https://pypi.org/project/profanity-check/
!pip install alt-profanity-check
!pip install sklearn --upgrade
#resource for sentiment: https://realpython.com/python-nltk-sentiment-analysis/#using-nltks-pre-trained-sentiment-analyzer
!pip install nltk
from profanity_check import predict
import nltk
nltk.download([
"names",
"stopwords",
"state_union",
"twitter_samples",
"movie_reviews",
"averaged_perceptron_tagger",
"vader_lexicon",
"punkt"])
from nltk.sentiment import SentimentIntensityAnalyzer
%matplotlib notebook
from google.colab import drive
drive.mount('/content/drive')
!pip install tensorflow-io
!pip install kafka-python
import os
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_io as tfio
!curl -sSOL https://downloads.apache.org/kafka/3.6.1/kafka_2.13-3.6.1.tgz
!tar -xzf kafka_2.13-3.6.1.tgz
!./kafka_2.13-3.6.1/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.6.1/config/zookeeper.properties
!./kafka_2.13-3.6.1/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.6.1/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10
!./kafka_2.13-3.6.1/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic ah-tweets-train
!./kafka_2.13-3.6.1/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 2 --topic ah-tweets-test

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.6/450.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-multiflow: filename=scikit_multiflow-0.5.3-cp310-cp310-linux_x86_64.whl size=1254678 sha256=3fc32720c6d9c99d0bbe486be3e2b57c5500bdca302075ca71685a286ee501ec
  Stored in directory: /root/.cache/pip/wheels/6e/1b/56/45b17a6cf203d98000a45976cb0dd0c4c3f11960e6a505f231
Successfully built scikit-multiflow
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 30.2 MB/s eta 0:00:00
  Created wheel for alt-profanity-check: filename=alt_profanity_check-1.3.2-py3-none-any.whl size=1866996 sha256=d4f1f5d9eb2039ed7092bc22cd5b3018e9186f6f403c593c27a8c42c8642035b
  Stored in directory: /root/.cache/pip/wheels/b3/b9/e1/e1ace2573792813935cd59a2f8a0cecc807bd2dbf69327a0b3
Successfully built alt-profanity-check
  Attempting uninstall: s

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package state_union to /root/nltk_data...
[nltk_data]   Unzipping corpora/state_union.zip.
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 82.1 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.34.0
    Uninstalling tensorflow-io-gcs-filesystem-0.34.0:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.34.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 4.3 MB/s eta 0:00:00
Waiting for 10 secs until kafka and zookeeper services are up and running
Created topic ah-tweets-train.
Created topic ah-tweets-test.


In [2]:
def error_callback(exc):
    raise Exception('Error while sendig data to kafka: {0}'.format(str(exc)))

def write_to_kafka(topic_name, items):
  count=0
  producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'])
  for message, key in items:
    producer.send(topic_name, key=key.encode('utf-8'), value=message.encode('utf-8')).add_errback(error_callback)
    count+=1
  producer.flush()
  print("Wrote {0} messages into topic: {1}".format(count, topic_name))

def decode_kafka_item(item):
  message = tf.io.decode_csv(item.message, [[0.0] for i in range(1)])
  key = tf.strings.to_number(item.key)
  return (message, key)

def decode_kafka_test_item(raw_message, raw_key):
  message = tf.io.decode_csv(raw_message, [[0.0] for i in range(1)])
  key = tf.strings.to_number(raw_key)
  return (message, key)

def decode_kafka_online_item(raw_message, raw_key):
  message = tf.io.decode_csv(raw_message, [[0.0] for i in range(1)])
  key = tf.strings.to_number(raw_key)
  return (message, key)

from sklearn import preprocessing

# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd

from collections.abc import Sequence
# Split into train/test
from sklearn.model_selection import train_test_split
# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(target_type, Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)

Tweet Feature Extraction And Data Cleaning/Processing

In [3]:
ah_tweets = pd.read_csv("/content/drive/MyDrive/Master's Project/hatespeech_text_label_vote_RESTRICTED_100K_v2.csv")
#had to fix dataframe column names
ah_tweets = ah_tweets.rename(columns={"Tweet text  Label   Votes for the majority label": "Tweet text", "Unnamed: 1": "Label", "Unnamed: 2": "Votes for the majority label"})
#last column not needed
ah_tweets.drop(ah_tweets.columns[[2]], axis=1, inplace=True)
#"spam" labels not needed
ah_tweets = ah_tweets[ah_tweets['Label'] != "spam"]

In [4]:
ah_tweets

,Tweet text,Label
1,RT @Papapishu: Man it would fucking rule if we...,abusive
2,It is time to draw close to Him &#128591;&#127...,normal
3,if you notice me start to act different or dis...,normal
4,"Forget unfollowers, I believe in growing. 7 ne...",normal
5,RT @Vitiligoprince: Hate Being sexually Frustr...,abusive
...,...,...
99991,RT @shangros: my fucking queen https://t.co/wa...,abusive
99992,#Osteporosis treated with #PEMF - rebuild bone...,normal
99993,@LGUSAMobile why does my phone screen keeps fl...,normal
99994,#bigdata vs. #reality ... but equally applies ...,normal


In [6]:
ah_tweets["@ count"] = ah_tweets['Tweet text'].str.count('@')
#Remove @'s from tweet text
ah_tweets['Tweet text'] = ah_tweets['Tweet text'].str.replace('@[\S]*', '')

ah_tweets['Hashtag count'] = ah_tweets['Tweet text'].str.count('#')
#Remove #'s from tweet text
ah_tweets['Tweet text'] = ah_tweets['Tweet text'].str.replace('#[\S]*','')

ah_tweets['Retweet'] = ah_tweets['Tweet text'].str.contains('RT')
#Remove RT's from tweet text
ah_tweets['Tweet text'] = ah_tweets['Tweet text'].str.replace('RT', '')

ah_tweets['Url count'] = ah_tweets['Tweet text'].str.count('http[\S]*')
#Remove URL's from tweet text
ah_tweets['Tweet text'] = ah_tweets['Tweet text'].str.replace('http[\S]*','')

ah_tweets['Uppercase word count'] = ah_tweets['Tweet text'].apply(lambda x: sum(map(str.isupper, x.split())))

ah_tweets['Num sentences'] = ah_tweets['Tweet text'].apply(lambda x: len(nltk.sent_tokenize(x)))

ah_tweets['temp'] = ah_tweets['Tweet text'].apply(lambda x: nltk.sent_tokenize(x))
ah_tweets['Avg sentence len'] = ah_tweets['temp'].apply(lambda x: pd.Series(x).apply(lambda y: len(nltk.word_tokenize(y))).mean())
ah_tweets.drop(ah_tweets.columns[[8]], axis=1, inplace=True)

#Got rid of ah_tweets with no text information after data cleaning/processing
ah_tweets['Text empty'] = ah_tweets['Tweet text'].apply(lambda x: x.isspace())
ah_tweets = ah_tweets[ah_tweets['Text empty'] != True]
ah_tweets.drop(ah_tweets.columns[[9]], axis=1, inplace=True)
ah_tweets['Text empty'] = ah_tweets['Tweet text'].apply(lambda x: len(x) == 0)
ah_tweets = ah_tweets[ah_tweets['Text empty'] != True]
ah_tweets.drop(ah_tweets.columns[[9]], axis=1, inplace=True)

<ipython-input-6-e4e02b169367>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  ah_tweets['Tweet text'] = ah_tweets['Tweet text'].str.replace('@[\S]*', '')
<ipython-input-6-e4e02b169367>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  ah_tweets['Tweet text'] = ah_tweets['Tweet text'].str.replace('#[\S]*','')
<ipython-input-6-e4e02b169367>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  ah_tweets['Tweet text'] = ah_tweets['Tweet text'].str.replace('http[\S]*','')
<ipython-input-6-e4e02b169367>:22: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ah_tweets['Avg sentence len'] = ah_tweets['temp'].apply(lambda x: pd.Series(x).apply(lambda y: len(nltk.word_tokenize(y))).mean())
<ipython-input-6-e4e02b169367>:28: SettingWithCopyW

In [7]:
sia = SentimentIntensityAnalyzer() #Sentiment is 'compound' value from -1 to 1
ah_tweets['Sentiment'] = ah_tweets['Tweet text'].apply(lambda x: sia.polarity_scores(x)['compound'])

#Create temperorary parts of speech column
from collections import Counter
ah_tweets['temp'] = ah_tweets['Tweet text'].apply(lambda x: nltk.pos_tag(x.split()))
ah_tweets['temp'] = ah_tweets['temp'].apply(lambda x: Counter(tag for word,tag in x))

#Counting all adjectives
ah_tweets['Adjective count'] = ah_tweets['temp'].apply(lambda x: x['JJ'])
ah_tweets['Adjective count'] = ah_tweets['Adjective count'] + ah_tweets['temp'].apply(lambda x: x['JJR'])
ah_tweets['Adjective count'] = ah_tweets['Adjective count'] + ah_tweets['temp'].apply(lambda x: x['JJS'])

#Counting all verbs
ah_tweets['Verb count'] = ah_tweets['temp'].apply(lambda x: x['VB'])
ah_tweets['Verb count'] = ah_tweets['Verb count'] + ah_tweets['temp'].apply(lambda x: x['VBP'])
ah_tweets['Verb count'] = ah_tweets['Verb count'] + ah_tweets['temp'].apply(lambda x: x['VBD'])
ah_tweets['Verb count'] = ah_tweets['Verb count'] + ah_tweets['temp'].apply(lambda x: x['VBG'])
ah_tweets['Verb count'] = ah_tweets['Verb count'] + ah_tweets['temp'].apply(lambda x: x['VBN'])
ah_tweets['Verb count'] = ah_tweets['Verb count'] + ah_tweets['temp'].apply(lambda x: x['VBZ'])

#Counting all adverbs
ah_tweets['Adverb count'] = ah_tweets['temp'].apply(lambda x: x['RB'])
ah_tweets['Adverb count'] = ah_tweets['Adverb count'] + ah_tweets['temp'].apply(lambda x: x['RBR'])
ah_tweets['Adverb count'] = ah_tweets['Adverb count'] + ah_tweets['temp'].apply(lambda x: x['RBS'])

#Remove temperoary parts of speech column
ah_tweets.drop(ah_tweets.columns[[10]], axis=1, inplace=True)

#Remove punctuation from tweet text
ah_tweets['Tweet text'] = ah_tweets['Tweet text'].str.replace('[^\w\s]', '')
#Remove numbers from tweet text
ah_tweets['Tweet text'] = ah_tweets['Tweet text'].str.replace('\d', '')
#Remove non-letters from tweet text
ah_tweets['Tweet text'] = ah_tweets['Tweet text'].str.replace('[^a-zA-Z]', ' ')

#Got rid of ah_tweets with no text information after data cleaning/processing
ah_tweets['Text empty'] = ah_tweets['Tweet text'].apply(lambda x: x.isspace())
ah_tweets = ah_tweets[ah_tweets['Text empty'] != True]
ah_tweets.drop(ah_tweets.columns[[13]], axis=1, inplace=True)
ah_tweets['Text empty'] = ah_tweets['Tweet text'].apply(lambda x: len(x) == 0)
ah_tweets = ah_tweets[ah_tweets['Text empty'] != True]
ah_tweets.drop(ah_tweets.columns[[13]], axis=1, inplace=True)

ah_tweets['Profanity count'] = ah_tweets['Tweet text'].str.replace('\W', ' ').str.split()
ah_tweets['Profanity count'] = ah_tweets['Profanity count'].apply(lambda x: sum(predict(x)))

ah_tweets['Avg word len'] = ah_tweets['Tweet text'].apply(lambda x: sum(list(map(len,x.split()))) / len(x.split()))

<ipython-input-7-b4c6cbbd29cf>:31: FutureWarning: The default value of regex will change from True to False in a future version.
  ah_tweets['Tweet text'] = ah_tweets['Tweet text'].str.replace('[^\w\s]', '')
<ipython-input-7-b4c6cbbd29cf>:33: FutureWarning: The default value of regex will change from True to False in a future version.
  ah_tweets['Tweet text'] = ah_tweets['Tweet text'].str.replace('\d', '')
<ipython-input-7-b4c6cbbd29cf>:35: FutureWarning: The default value of regex will change from True to False in a future version.
  ah_tweets['Tweet text'] = ah_tweets['Tweet text'].str.replace('[^a-zA-Z]', ' ')
<ipython-input-7-b4c6cbbd29cf>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ah_tweets.drop(ah_tweets.columns[[13]], axis=1, inplace=True)
<ipython-input-7-b4c6cbbd29cf>:41: SettingWithCo

In [ ]:
ah_tweets_copy = ah_tweets.copy()
ah_tweets

In [8]:
ah_tweets.drop(ah_tweets.columns[[0]], axis=1, inplace=True)
category = encode_text_index(ah_tweets, "Label")
#'abusive' ah_tweets label = 0
#'hateful' ah_tweets label = 1
#'normal' ah_tweets label = 2
encode_numeric_zscore(ah_tweets, "@ count")
encode_numeric_zscore(ah_tweets, "Hashtag count")
encode_text_dummy(ah_tweets, "Retweet")
encode_numeric_zscore(ah_tweets, "Url count")
encode_numeric_zscore(ah_tweets, "Uppercase word count")
encode_numeric_zscore(ah_tweets, "Num sentences")
encode_numeric_zscore(ah_tweets, "Adjective count")
encode_numeric_zscore(ah_tweets, "Verb count")
encode_numeric_zscore(ah_tweets, "Adverb count")
encode_numeric_zscore(ah_tweets, "Profanity count")
ah_tweets

Create A/H Tweets Data Stream

In [13]:
x_ah = ah_tweets.iloc[:,1:]
y_ah = ah_tweets['Label']
x_train_ah, x_test_ah, y_train_ah, y_test_ah = train_test_split(x_ah, y_ah, test_size=0.3, random_state=42)

# The labels are set as the kafka message keys so as to store data
# in multiple-partitions. Thus, enabling efficient data retrieval
# using the consumer groups.
x_train_ah = list(filter(None, x_train_ah.to_csv(index=False).split("\n")[1:]))
y_train_ah = list(filter(None, y_train_ah.to_csv(index=False).split("\n")[1:]))

x_test_ah = list(filter(None, x_test_ah.to_csv(index=False).split("\n")[1:]))
y_test_ah = list(filter(None, y_test_ah.to_csv(index=False).split("\n")[1:]))

In [14]:
write_to_kafka("ah-tweets-train", zip(x_train_ah, y_train_ah))
write_to_kafka("ah-tweets-test", zip(x_test_ah, y_test_ah))

Wrote 60027 messages into topic: ah-tweets-train
Wrote 25727 messages into topic: ah-tweets-test


Consume A/H Tweets From Data Stream

In [ ]:
online_train_ah_tweets = tfio.experimental.streaming.KafkaBatchIODataset(
    topics=["ah-tweets-train"],
    group_id="online-train-ah",
    servers="127.0.0.1:9092",
    stream_timeout=10000, # in milliseconds, to block indefinitely, set it to -1.
    configuration=[
        "session.timeout.ms=7000",
        "max.poll.interval.ms=8000",
        "auto.offset.reset=earliest"
    ],
)

test_ah_tweets = tfio.experimental.streaming.KafkaGroupIODataset(
    topics=["ah-tweets-test"],
    group_id="test-ah",
    servers="127.0.0.1:9092",
    stream_timeout=10000,
    configuration=[
        "session.timeout.ms=7000",
        "max.poll.interval.ms=8000",
        "auto.offset.reset=earliest"
    ],
)

In [ ]:
"""test_ah_tweets = test_ah_tweets.map(decode_kafka_test_item)
test_ah_tweets = test_ah_tweets.batch(BATCH_SIZE)"""

for mini_ds in online_train_ah_tweets:
  mini_ds = mini_ds.shuffle(buffer_size=32)
  mini_ds = mini_ds.map(decode_kafka_online_item)
  mini_ds = mini_ds.batch(32)
  if len(mini_ds) > 0:
    print(mini_ds)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=

Aggressive Tweets

In [ ]:
#combine abusive and hateful tweets into aggressive for later comparison with other work
aggressive_tweets = ah_tweets_copy
aggressive_tweets = aggressive_tweets.replace("abusive","aggressive")
aggressive_tweets = aggressive_tweets.replace("hateful","aggressive")
aggressive_tweets

In [ ]:
#check to make sure only aggressive and normal tweet labels exist
pd.unique(aggressive_tweets['Label'])

In [ ]:
aggressive_tweets.drop(aggressive_tweets.columns[[0]], axis=1, inplace=True)
category = encode_text_index(aggressive_tweets, "Label")
#'aggressive' tweets label = 0
#'normal' tweets label = 1
encode_numeric_zscore(aggressive_tweets, "@ count")
encode_numeric_zscore(aggressive_tweets, "Hashtag count")
encode_text_dummy(aggressive_tweets, "Retweet")
encode_numeric_zscore(aggressive_tweets, "Url count")
encode_numeric_zscore(aggressive_tweets, "Uppercase word count")
encode_numeric_zscore(aggressive_tweets, "Num sentences")
encode_numeric_zscore(aggressive_tweets, "Adjective count")
encode_numeric_zscore(aggressive_tweets, "Verb count")
encode_numeric_zscore(aggressive_tweets, "Adverb count")
encode_numeric_zscore(aggressive_tweets, "Profanity count")
aggressive_tweets

Create Aggressive Tweets Data Stream

In [ ]:
x_agg = aggressive_tweets.iloc[:,1:]
y_agg = aggressive_tweets['Label']
x_train_agg, x_test_agg, y_train_agg, y_test_agg = train_test_split(x_agg, y_agg, test_size=0.3, random_state=42)

# The labels are set as the kafka message keys so as to store data
# in multiple-partitions. Thus, enabling efficient data retrieval
# using the consumer groups.
x_train_agg = list(filter(None, x_train_agg.to_csv(index=False).split("\n")[1:]))
y_train_agg = list(filter(None, y_train_agg.to_csv(index=False).split("\n")[1:]))

x_test_agg = list(filter(None, x_test_agg.to_csv(index=False).split("\n")[1:]))
y_test_agg = list(filter(None, y_test_agg.to_csv(index=False).split("\n")[1:]))

In [ ]:
write_to_kafka("agg-tweets-train", zip(x_train_agg, y_train_agg))
write_to_kafka("agg-tweets-test", zip(x_test_agg, y_test_agg))